In [3]:
# install.packages(c("RDFTensor", "dplyr"))

library(dplyr)
library(RDFTensor)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [1]:
setwd("D://LA//ATI Data")

In [4]:
Auth <- read.table(file="Summaries//Just_Auth//LA.txt", header=T, sep=",")
Auth <- Auth[, -1]

In [6]:
head(Auth)
dim(Auth)
Auth %>% mutate(across(where(is.factor), as.character)) -> Auth

str(Auth)

UserName,SrcDevice,DstDevice,AuthType,Failure,DailyCount
User035855,Comp808475,Comp081330,TGS,0,17
Comp655251$,Comp655251,ActiveDirectory,NetworkLogon,0,350
User762066,Comp306129,ActiveDirectory,TGS,0,22
User384215,Comp095190,EnterpriseAppServer,NetworkLogon,0,35
User043263,Comp883307,Comp384394,TGS,0,2
User631552,Comp621781,Comp915658,NetworkLogon,0,1


[1] 15953681        6

'data.frame':	15953681 obs. of  6 variables:
 $ UserName  : chr  "User035855" "Comp655251$" "User762066" "User384215" ...
 $ SrcDevice : chr  "Comp808475" "Comp655251" "Comp306129" "Comp095190" ...
 $ DstDevice : chr  "Comp081330" "ActiveDirectory" "ActiveDirectory" "EnterpriseAppServer" ...
 $ AuthType  : chr  "TGS" "NetworkLogon" "TGS" "NetworkLogon" ...
 $ Failure   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ DailyCount: int  17 350 22 35 2 1 14 78 2 4 ...


### User179746

In [7]:
U17 <- Auth[Auth$UserName=='User179746', ]

In [35]:
df<- as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Fail=U17$Failure))

In [36]:
head(df)
dim(df)

Src,Dst,Fail
Comp599155,None,0
Comp599155,None,0
Comp599155,None,0
Comp679250,Comp599155,0
Comp547871,ActiveDirectory,1
Comp679250,Comp599155,0


[1] 57  3

In [37]:
SrcT <- tapply(df$Src, df$Src)
DstT <- tapply(df$Dst, df$Dst)

dm <- as.matrix(cbind(SrcT, DstT, df$Fail))

In [54]:
head(dm)

head(df)

SrcT,DstT,
3,25,1
3,25,1
3,25,1
4,14,1
2,1,2
4,14,1


Src,Dst,Fail
Comp599155,None,0
Comp599155,None,0
Comp599155,None,0
Comp679250,Comp599155,0
Comp547871,ActiveDirectory,1
Comp679250,Comp599155,0


In [53]:
subs=matrix(c(5,1,1,
              3,1,2,
              1,1,3,
              2,1,3,
              4,1,3,
              6,1,3,
              1,1,4,
              2,1,4,
              4,1,4,
              6,1,4,
              1,2,1,
              3,2,1,
              5,2,1),byrow=TRUE,ncol=3)
X=list(subs=subs,vals=rep(1,nrow(subs)),size=c(6,2,4))

subs
X

5,1,1
3,1,2
1,1,3
2,1,3
4,1,3
6,1,3
1,1,4
2,1,4
4,1,4
6,1,4
1,2,1


5,1,1
3,1,2
1,1,3
2,1,3
4,1,3
6,1,3
1,1,4
2,1,4
4,1,4
6,1,4
1,2,1


In [50]:
M <- list(subs=dm, vals=rep(1, nrow(dm)), size=c(length(unique(SrcT)), length(unique(DstT)), 2))
invisible(M)

tfm<- getTensor(trp=dm)

In [56]:
set.seed(5)
P1 <- cp_apr(M,2,opts=list(alg='mu'))

print(P1$M)

[1] "CP_APR: mu"
[1] " Iter    1: Inner Its = 23 KKT violation = 1.335418e-01, nViolations =  0"
[1] " Iter    2: Inner Its =  8 KKT violation = 6.241433e-07, nViolations =  0"
[1] " Iter    3: Inner Its =  3 KKT violation = 6.241433e-07, nViolations =  0"
[1] "Exiting because all subproblems reached KKT tol."


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -2.925011e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 6.2414330e-07"
[1] " Total inner iterations = 34"
[1] " Total execution time = 0.21 secs"
$lambda
[1] 48  9

$u
$u[[1]]
              [,1]         [,2]
[1,] 2.600441e-128 1.111111e-01
[2,]  6.250000e-02 3.213536e-09
[3,]  9.375000e-01 6.934926e-08
[4,] 9.044691e-118 8.888888e-01

$u[[2]]
               [,1]         [,2]
 [1,]  4.166667e-02 3.470756e-34
 [2,]  4.166667e-02 2.555932e-14
 [3,]  2.083333e-02 1.272722e-14
 [4,]  2.083333e-02 6.725709e-15
 [5,]  4.166667e-02 3.273877e-14
 [6,]  2.083333e-02 1.162181e-14
 [7,]  2.083333e-02 6.695594e-15
 [8,]  4.166667e-02 1.690307e-14
 [9,]  2.083333e-02 1.952144e-14
[10,]  2.083333e-02 1.602380e-14
[11,]  2.083333e-02 3.523004e-14
[12,]  4.166667e-02 4.726771e-15
[13,] 3.393511e-156 2.222222e-01
[14,] 9.979302e-156 7.777778e-01
[15,]  4.166667e-02 1.621977e-14
[16,]  1.250000e-01 4.

In [67]:
P1_2<- cp_apr(M,2,opts=list(alg='pdnr'))

print(P1_2$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 62, KKT viol = 2.76e+00, obj = -5.71527178e+01, nz: 7, time:0.58"
[1] "   2. Ttl Inner Its: 134, KKT viol = 2.63e+00, obj = -3.39026314e+01, nz: 11, time:0.75"
[1] "   3. Ttl Inner Its: 42, KKT viol = 9.96e-01, obj = -2.95267136e+01, nz: 12, time:0.76"
[1] "   4. Ttl Inner Its: 53, KKT viol = 3.13e-02, obj = -2.92786951e+01, nz: 26, time:0.80"
[1] "   5. Ttl Inner Its: 62, KKT viol = 9.74e-03, obj = -2.92501049e+01, nz: 29, time:0.83"
[1] "   6. Ttl Inner Its: 31, KKT viol = 6.77e-05, obj = -2.92501049e+01, nz: 29, time:0.84"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -2.925010e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 6.7709872e-05"
[1] " Total inner iterations = 384"
[1] " Total execution time = 0.84 secs"
$lambda
[1]  8.999972 47.999815

$u
$u[[1]]
          [,1]       [,2]
[1,] 0.1111111 0.00000000
[2,] 0.0000000 0.06249999
[3,] 0.0000000 0.93750001
[4,] 0.8888889 0.00000000

$u[[2]]
           [,1]       [,2]
 [1,] 0.0000000 0.04166741
 [2,] 0.0000000 0.04166740
 [3,] 0.0000000 0.02083371
 [4,] 0.0000000 0.02083370
 [5,] 0.0000000 0.04166401
 [6,] 0.0000000 0.02083370
 [7,] 0.0000000 0.02083370
 [8,] 0.0000000 0.04166740
 [9,] 0.0000000 0.02083370
[10,] 0.0000000 0.02083351
[11,] 0.0000000 0.02083371
[12,] 0.0000000 0.04166740
[13,] 0.2222223 0.00000000
[14,] 0.7777777 0.00000000
[15,] 0.0000000 0.04166676
[16,] 0.0000000 0.12499877
[17,] 0.0000000 0.02083347
[18,] 0.0000000 0.04166741
[19,] 0.0000000 0.04166740
[20,] 0.0000000 0.0416674

In [92]:
P1_3<- cp_apr(M,2,opts=list(alg='pqnr'))

print(P1_3$M)

[1] "CP_PQNR (alternating Poisson regression using quasi-Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "-lno=1-i=1--jj:1----tmp_rho:381.795996,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.991961"
[1] "-lno=2-i=2--jj:1----tmp_rho:0.064525,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:0.187459"
[1] "-lno=3-i=3--jj:1----tmp_rho:13.185215,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:0.069235"
[1] "-lno=4-i=4--jj:1----tmp_rho:69.083059,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.051902"
[1] "-lno=5-i=5--jj:1----tmp_rho:262.168948,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:0.008489"
[1] "-lno=6-i=6--jj:1----tmp_rho:11892.169051,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:0.000717"
[1] "-lno=7-i=1--jj:2----tmp_rho:44.607092,lbfgsPos:1,nSizeLBFGS:3, kkt_violation:0.870445"
[1] "-lno=8-i=2--jj:2----tmp_rho:0.001608,lbfgsPos:2,nSizeLBFGS:3, kkt_violation:37.633855"
[1] "-lno=9-i=3--jj:2----tmp_rho:0.003312,lbfgsPos:3,nSizeLBFGS:3, kkt_violation:0.735006"
[1] "-lno=10-i=4--jj:2----tmp_rho:274.045753,lbfgs

Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -2.925010e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 1.5062234e-05"
[1] " Total inner iterations = 489"
[1] " Total execution time = 0.75 secs"
$lambda
[1]  8.999982 47.999910

$u
$u[[1]]
          [,1]       [,2]
[1,] 0.1111111 0.00000000
[2,] 0.0000000 0.06250023
[3,] 0.0000000 0.93749977
[4,] 0.8888889 0.00000000

$u[[2]]
           [,1]       [,2]
 [1,] 0.0000000 0.04166672
 [2,] 0.0000000 0.04166672
 [3,] 0.0000000 0.02083337
 [4,] 0.0000000 0.02083337
 [5,] 0.0000000 0.04166672
 [6,] 0.0000000 0.02083337
 [7,] 0.0000000 0.02083337
 [8,] 0.0000000 0.04166672
 [9,] 0.0000000 0.02083337
[10,] 0.0000000 0.02083337
[11,] 0.0000000 0.02083337
[12,] 0.0000000 0.04166672
[13,] 0.2222261 0.00000000
[14,] 0.7777739 0.00000000
[15,] 0.0000000 0.04166672
[16,] 0.0000000 0.12499970
[17,] 0.0000000 0.02083337
[18,] 0.0000000 0.04166672
[19,] 0.0000000 0.04166672
[20,] 0.0000000 0.0416667

In [95]:
P1_4<- cp_apr(M,2,opts=list(alg='kappatol'))

print(P1_4$M)

            ## The alg is available in the doc but not yet supported as of current R -v

ERROR: Error in cp_apr(M, 2, opts = list(alg = "kappatol")): invalid alg:kappatol, possible values are mu,pdnr,pqnr


**CP_NMU implementation [ Fail ]**

In [66]:
set.seed(15)

P2 <- cp_nmu(M, R=2, opts=list(alg='pdnr'))

[1] 1 2 3
[1] "Nonnegative PARAFAC:"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


Warning message in sqrt(normX^2 + ktensor_norm(P)^2 - 2 * tt_innerprod(P, X)):
"NaNs produced"

[1] " Iter  1: fit = NaN fitdelta =     NaN, itime=0.00"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


Warning message in sqrt(normX^2 + ktensor_norm(P)^2 - 2 * tt_innerprod(P, X)):
"NaNs produced"

[1] " Iter  2: fit = NaN fitdelta =     NaN, itime=0.01"


ERROR: Error in if ((iter > 1) && (fitchange < fitchangetol)) {: missing value where TRUE/FALSE needed


In [73]:
tnsr=getTensor(dm)

[1] "The modes:"
[1]  4  4 25


**CP_01 implementation [ Fail ]**

In [87]:
df<- as.matrix(as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Auth=U17$AuthType)))

head(df, 10)
class(df) 

Src,Dst,Auth
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp679250,Comp599155,NetworkLogon
Comp547871,ActiveDirectory,NetworkLogon
Comp679250,Comp599155,RemoteInteractive
Comp679250,Comp599155,NetworkLogon
Comp215429,Comp599155,NetworkLogon
Comp599155,Comp936659,NetworkLogon
Comp599155,Comp422203,NetworkLogon


[1] "matrix"

In [91]:
tnsr=getTensor(df)
subs=getTnsrijk(tnsr$X)
T<- list(subs=subs,vals=rep(1,nrow(subs)),size=c(length(unique(df[,1])), length(unique(df[,2])), length(unique(df[,3]))))
normT=sqrt(sum(T$vals))

set.seed(555)
P1<- cp_nmu(T, R=2)
res<- CP_R01(T, P1[[1]])

[1] "The modes:"
[1]  7  7 25
[1] 1 2 3
[1] "Nonnegative PARAFAC:"
[1] "n:1, calc spt_mttkrp..."
[1] "n:2, calc spt_mttkrp..."
[1] "n:3, calc spt_mttkrp..."


ERROR: Error in pracma::accumarray(newsubs, newvals, sz = newsiz): Argument 'sz' does not fit with 'subs'.


In [99]:
df<- as.matrix(as.data.frame(cbind(Src=U17$SrcDevice, Dst=U17$DstDevice, Auth=U17$AuthType)))

t1<- tapply(df[,1], df[,1])
t2<- tapply(df[,2], df[,2])
t3<- tapply(df[,3], df[,3])

dt<- as.matrix(cbind(t1,t2,t3))

head(dt, 10)
head(df, 10)

t1,t2,t3
3,25,1
3,25,1
3,25,1
4,14,2
2,1,2
4,14,3
4,14,2
1,14,2
3,21,2
3,10,2


Src,Dst,Auth
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp599155,None,InteractiveLogon
Comp679250,Comp599155,NetworkLogon
Comp547871,ActiveDirectory,NetworkLogon
Comp679250,Comp599155,RemoteInteractive
Comp679250,Comp599155,NetworkLogon
Comp215429,Comp599155,NetworkLogon
Comp599155,Comp936659,NetworkLogon
Comp599155,Comp422203,NetworkLogon


In [97]:
MT <- list(subs=dt, vals=rep(1, nrow(dt)), size=c(length(unique(dt[,1])), length(unique(dt[,2])), length(unique(dt[,3]))))

P<- cp_apr(MT,2,opts=list(alg='pdnr'))

print(P$M)

[1] 0
[1] "CP_PDNR (alternating Poisson regression using damped Newton)"
[1] "  Precomputing sparse index sets..."
[1] "done"
[1] "   1. Ttl Inner Its: 64, KKT viol = 7.56e+00, obj = -4.49466047e+01, nz: 7, time:0.10"
[1] "   2. Ttl Inner Its: 63, KKT viol = 2.58e+00, obj = -3.17956079e+01, nz: 9, time:0.13"
[1] "   3. Ttl Inner Its: 50, KKT viol = 3.64e-01, obj = -3.11874805e+01, nz: 9, time:0.16"
[1] "   4. Ttl Inner Its: 62, KKT viol = 2.53e-02, obj = -3.10041318e+01, nz: 9, time:0.21"
[1] "   5. Ttl Inner Its: 68, KKT viol = 4.14e-03, obj = -3.09800666e+01, nz: 9, time:0.25"
[1] "   6. Ttl Inner Its: 51, KKT viol = 6.20e-04, obj = -3.09787069e+01, nz: 9, time:0.28"
[1] "   7. Ttl Inner Its: 32, KKT viol = 9.45e-05, obj = -3.09787069e+01, nz: 9, time:0.30"


Warning message in sqrt(normX^2 + ktensor_norm(M)^2 - 2 * tt_innerprod(M, X)):
"NaNs produced"

[1] "==========================================="
[1] " Final log-likelihood = -3.097871e+01 "
[1] " Final least squares fit = NaN "
[1] " Final KKT violation = 9.4468094e-05"
[1] " Total inner iterations = 390"
[1] " Total execution time = 0.30 secs"
$lambda
[1] 40.9999 16.0008

$u
$u[[1]]
     [,1]       [,2]
[1,]    0 0.06249886
[2,]    0 0.18749657
[3,]    1 0.25001373
[4,]    0 0.49999085

$u[[2]]
            [,1]         [,2]
 [1,] 0.00000000 1.249938e-01
 [2,] 0.04878163 3.635095e-06
 [3,] 0.02439072 2.058409e-06
 [4,] 0.02439058 2.428417e-06
 [5,] 0.04877999 1.566669e-06
 [6,] 0.02438953 5.107033e-06
 [7,] 0.02438984 1.189445e-06
 [8,] 0.04878026 8.901282e-07
 [9,] 0.02438975 1.417697e-06
[10,] 0.02438995 9.128863e-07
[11,] 0.02439049 2.647400e-06
[12,] 0.04878162 3.668213e-06
[13,] 0.00000000 1.249938e-01
[14,] 0.00000000 4.374781e-01
[15,] 0.04878017 1.116548e-06
[16,] 0.14634121 1.536954e-06
[17,] 0.02438937 5.512389e-06
[18,] 0.04878004 1.441325e-06
[19,] 0.04878011 1.26820